![](https://imgur.com/J2wx5Pg)


One of the biggest problems for those who want to participate in Microsoft's competition is going to be the size of this massive dataset.With a train set of almost 9 million rows and 83 columns the first thing every Kaggler will notice is that we can't load the data in a single dataframe.While there are methods to get past this ( like using the chunksize option ) i thought it would be usefull to  load a part of the data and then decide which features we are going to load when we are going to train a model.

In [ ]:
import numpy as np
import pandas as pd
import gc  # Garbage Collector
number_of_rows = 2000000 # We are loading 2 million rows and skip the others.

def unique_columns(dataframe): # A function to drop all columns where all rows are the same.
    list_name = []
    temp = list(dataframe.nunique().values)
    for i in range(len(temp)):
        if temp[i] == dataframe.shape[0] or temp[i] == 1:
            list_name.append(dataframe.columns[i])
    print('Columns that were dropped:',list_name)
    return list_name

def get_top_features(fscore,df,model): # A function to get the features of a trained model with
    temp=[]                            # fscore greater than the fscore parameter.
    for i in range(len(model.feature_importances_)):
        if model.feature_importances_[i]>=fscore:
            temp.append(df.columns[i])
    print('Features with fscore greater than ',fscore,':',temp)
    return temp

train = pd.read_csv( '../input/train.csv' , skiprows =  lambda x: x > number_of_rows)
train.tail() 

The 'MachineIdentifier' feature is a unique ID for every machine so it's not going to add much information so we can drop this column.We can also drop all columns where all rows are the same.

In [ ]:
train.drop(unique_columns(train),axis = 1,inplace = True)

There are many categorical features in our dataset like 'ProductName' and 'EngineVersion'. We will have to use a Label Encoder hor those in order to be able to train a model.Before doing that let's see all the features that are of type 'object' (which means that are parsed as strings).

In [ ]:
train.select_dtypes(include = 'object').head()

Columns AppVersion,EngineVersion, AvSigVersion and OsVersion can be turned into an meaningfull number if we take out the dot.

In [ ]:
train.OsVer = train.OsVer.apply(func = lambda x:x.replace(".","")).astype(float)
train.AppVersion = train.AppVersion.apply(func = lambda x:x.replace(".","")).astype(float)
train.EngineVersion = train.EngineVersion.apply(func = lambda x:x.replace(".","")).astype(float)
train.AvSigVersion = train.AvSigVersion.apply(func = lambda x:x.replace(".","")).astype(float)

Census_OSEdition feature and 

> Features Census_OSEdition and Census_OSSkuName contain the same information throughout the dataset so we can drop one and keep the other.

In [ ]:
train.drop(['Census_OSEdition'],axis = 1, inplace = True)

We are going to turn the rest feature columns into categorical columns and let pandas encode them ( Label Encoder was slower).

In [ ]:
for column in train.select_dtypes(include = 'object').columns:
    print(column)
    train[column]=train[column].astype('category')
    train[column]=train[column].cat.codes

In [ ]:
import seaborn as sns
correlation_matrix = train.corr()
mask = np.zeros_like(correlation_matrix)
mask[np.triu_indices_from(mask)] = True 
sns.heatmap(correlation_matrix, mask = mask)

We are only interested in the last row of the correlation matrix.

In [ ]:
correlation_matrix

In [ ]:
y_train = train['HasDetections']
X_train = train.drop(['HasDetections'],axis=1)
del(train) # save some space
gc.collect()

We are training a simple XGBoost model to get the feature importances.

In [ ]:
import xgboost as xgb
from xgboost import plot_importance
xgbo = xgb.XGBClassifier()
xgbo.fit(X_train,y_train)
plot_importance(xgbo)

A more clean visualization.

In [ ]:
from matplotlib import pyplot
pyplot.bar(range(len(xgbo.feature_importances_)), xgbo.feature_importances_)
pyplot.show()
columns_to_be_used = get_top_features(fscore = 0.04 , df = X_train, model = xgbo)

In [ ]:
del(X_train)
del(y_train)
gc.collect()

Now we can load all rows and only the columns of the features with and fscore of 0.04 or greater.

In [ ]:
train = pd.read_csv( '../input/train.csv' , usecols = columns_to_be_used)


In [ ]:
train.shape